In [92]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [93]:
clean_arr = np.loadtxt("wifi_db/clean_dataset.txt")
noisy_arr = np.loadtxt("wifi_db/noisy_dataset.txt")

In [94]:
LABEL_COL = clean_arr.shape[1] - 1 # LABEL_COL = 7
DATASET_SIZE = clean_arr.shape[0]
NUM_FOLDS = 10
labels = [1, 2, 3, 4]
noisy_arr

array([[-59., -53., -51., ..., -79., -87.,   4.],
       [-66., -53., -59., ..., -81., -79.,   1.],
       [-41., -57., -63., ..., -66., -65.,   2.],
       ...,
       [-57., -54., -56., ..., -79., -82.,   1.],
       [-56., -52., -50., ..., -85., -88.,   3.],
       [-46., -54., -47., ..., -80., -73.,   3.]])

In [115]:
class Node():
    # if node is leaf, value is used to store label
    # if node is parent, value is used to store the split_value AND
    # attr is used to store the attribute that is split on
    def __init__(self, **kwargs):
        self.label = kwargs.get('label', -1)
        self.value = kwargs.get('value', 0)
        self.attr = kwargs.get('attr', 0)
        self.left = kwargs.get('left', None)
        self.right = kwargs.get('right', None)
    
    def set_left(self, left):
        self.left = left
    
    def set_right(self, right):
        self.right = right
    
    def is_leaf(self):
        return self.left == None and self.right == None
    
    def is_parent_of_leafs(self):
        return self.is_leaf(self.left) and self.is_leaf(self.right)

In [96]:
def calc_label_instances(dataset, label):
    return np.count_nonzero(dataset[:, LABEL_COL] == label)


In [97]:
# since the log operation may return nan (eg log 0), replace all nan occurences with 0
def entropy(dataset):
    res = np.zeros(len(labels))
    for i in range(len(labels)):
        res[i] = calc_label_instances(dataset, labels[i])
    res = res / DATASET_SIZE
    log_res = np.where(res > 0, np.log2(res), res)
    res = res * log_res
    return - np.sum(res)

In [98]:
# calculate info gain after splitting data set, assume dataset is sorted already
def remainder(dataset, split_index):
    left_info_gain = (split_index + 1) / DATASET_SIZE * entropy(dataset[: (split_index + 1)])
    right_info_gain = (DATASET_SIZE - split_index)/DATASET_SIZE * entropy(dataset[(split_index + 1):])
    return left_info_gain + right_info_gain


def calc_info_gain(start_entropy, dataset, split_index):
    return start_entropy - remainder(dataset, split_index)

In [99]:
def get_mid(a, b):
    return (a + b) / 2

def find_split(trng_data):
    split_attribute = -1
    split_value = 0
    split_index = 0
    overall_highest_info_gain = 0
    # start_entropy calculated at the start, saving recomputation
    start_entropy = entropy(trng_data)
    dataset_size, _ = trng_data.shape
    final_sorted_dataset = trng_data
    
    # sort all data amongst columns
    # loop through each attribute
    for i in range(LABEL_COL):
        sorted_dataset = trng_data[trng_data[:, i].argsort()]
#         print('sorted dataset')
#         print(sorted_dataset[:, [i, LABEL_COL]]);
        current_split_value = 0
        feature_highest_info_gain = 0
        current_split_index = 0

        for j in range(dataset_size - 1):
            if sorted_dataset[j][LABEL_COL] == sorted_dataset[j + 1][LABEL_COL]: continue

            mid = get_mid(sorted_dataset[j][i], sorted_dataset[j + 1][i])
            info_gain = calc_info_gain(start_entropy, sorted_dataset, j)

            if (not math.isnan(info_gain)) and info_gain > feature_highest_info_gain:
                current_split_value = mid
                feature_highest_info_gain = info_gain
                current_split_index = j

        if overall_highest_info_gain < feature_highest_info_gain:
            overall_highest_info_gain = feature_highest_info_gain
            split_attribute = i
            split_value = current_split_value
            split_index = current_split_index
            final_sorted_dataset = sorted_dataset
#         print("Feature " + str(i) + " info gain: " + str(feature_highest_info_gain))

#     print("Start Entropy: " + str(start_entropy))
#     print("Info Gain: " + str(overall_highest_info_gain))
        
    return (split_attribute, split_value, split_index, final_sorted_dataset)

find_split(clean_arr)

<ipython-input-97-1016b69407cb>:7: RuntimeWarning: divide by zero encountered in log2
  log_res = np.where(res > 0, np.log2(res), res)


(0,
 -54.5,
 1011,
 array([[-74., -62., -66., ..., -89., -87.,   1.],
        [-73., -62., -66., ..., -87., -89.,   1.],
        [-73., -61., -65., ..., -89., -90.,   1.],
        ...,
        [-10., -58., -54., ..., -72., -73.,   2.],
        [-10., -58., -62., ..., -76., -74.,   2.],
        [-10., -57., -57., ..., -82., -72.,   2.]]))

In [100]:
def decision_tree_learning(training_dataset, depth):
    first_label = training_dataset[0][LABEL_COL]
    if np.all(training_dataset[:, LABEL_COL] == first_label):
        return (Node(label=first_label), depth)
    else:
        split_attr, split_value, split_index, sorted_dataset = find_split(training_dataset)
        curr_node = Node(value=split_value, attr=split_attr)
        left_branch, left_depth = decision_tree_learning(sorted_dataset[:split_index + 1], depth + 1)
        right_branch, right_depth = decision_tree_learning(sorted_dataset[split_index + 1:], depth + 1)
        curr_node.set_left(left_branch)
        curr_node.set_right(right_branch)
    return (curr_node, max(left_depth, right_depth))

In [101]:
root, depth = decision_tree_learning(clean_arr, 0)
# dirty_r, d = decision_tree_learning(noisy_arr, 0)
# print using py_plot
def print_nodes(node):
    if node.left != None:
        print_nodes(node.left)
    if node.attr != None:
        print(node.attr)
    print(node.value)
    if node.right != None:
        print_nodes(node.right)
        
print(depth)

<ipython-input-97-1016b69407cb>:7: RuntimeWarning: divide by zero encountered in log2
  log_res = np.where(res > 0, np.log2(res), res)


484


In [102]:
# print_nodes(root)

In [112]:
confusion_matrix = np.zeros(16).reshape((4, 4))
# take class 1 as positive

def traverse_tree(root, datapoint):
    if(root.left is None and root.right is None):
        return root.label
    attr = root.attr
    split_val = root.value
    if(datapoint[attr] <= split_val):
        return traverse_tree(root.left, datapoint)
    else:
        return traverse_tree(root.right, datapoint)
    

def evaluate(test_db, trained_tree):
    # returns accuracy of tree
    for i in range(len(test_db)):
        classified_label = traverse_tree(trained_tree, test_db[i])
        actual_label = test_db[i][LABEL_COL]
        confusion_matrix[int(actual_label) - 1,int(classified_label) - 1] += 1
    
    true_positive = confusion_matrix[0,0] 
    true_negative = confusion_matrix[1,1] + confusion_matrix[2,2] + confusion_matrix[3,3]
    all_values = len(test_db)
    accurately_classified_data = true_positive + true_negative
    accuracy = accurately_classified_data / all_values
    
    return (confusion_matrix, accuracy)

confusion_matrix

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [ ]:
def calculate_recall(matrix, i):
    row = matrix[i]
    return matrix[i][i] / sum(row)

def calculate_precision(matrix, i):
    col = matrix[:, i]
    return matrix[i][i] / sum(col)

def cross_validation(data):
    recall_per_class = np.zeros(4)
    precision_per_class = np.zeros(4)
    f1_per_class = np.zeros(4)
    sum_accuracy = 0
    sum_confusion_matrix = np.zeros(16).reshape((4, 4))
    np.random.shuffle(data)
    for i in range(NUM_FOLDS):
        begin = i * NUM_FOLDS
        end = (i + 1) * NUM_FOLDS + 1
        test_data = data[begin: end]
        trng_data = np.delete(data, slice(begin, end), axis=0)
#         print(trng_data)
        root, depth = decision_tree_learning(trng_data, 0)
        confusion_matrix, accuracy = evaluate(test_data, root)
        sum_accuracy += accuracy
        sum_confusion_matrix += confusion_matrix
        
        for i in range(len(labels)):
            recall_per_class[i] += calculate_recall(confusion_matrix, i)
            precision_per_class[i] += calculate_precision(confusion_matrix, i)
        
    average_recall_per_class = recall_per_class / NUM_FOLDS
    average_precision_per_class = precision_per_class / NUM_FOLDS
    f1_per_class = 2 * average_recall_per_class * average_precision_per_class / (average_recall_per_class + average_precision_per_class)
    avg_accuracy = sum_accuracy / NUM_FOLDS
    avg_confusion_matrix = sum_confusion_matrix / NUM_FOLDS
    return (avg_accuracy, average_recall_per_class, average_precision_per_class, f1_per_class, avg_confusion_matrix)

acc, recall, precision, f1, conf_matrix = cross_validation(clean_arr)
print(acc)
print(recall)
print(precision)
print(f1)
print(conf_matrix)

<ipython-input-97-1016b69407cb>:7: RuntimeWarning: divide by zero encountered in log2
  log_res = np.where(res > 0, np.log2(res), res)


In [ ]:
# pruning
# 1. Go through each internal node that are connected only to leaf nodes.
# 2. Turn each into a leaf node (with majority class label)
# 3. Evaluate pruned tree on validation set. Prune if accuracy higher than unpruned.

def prune(root):
    pass

